In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import sqlite3 
from pandasql import sqldf

pysqldf = lambda q: sqldf(q, globals())

from pandasql import sqldf, load_meat, load_births

In [98]:
sqladb = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", header=None)
sqladb.columns=['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', '-']

connection=sqlite3.connect(r"sqladb3.db")
cursor = connection.cursor()

query_create_table = '''CREATE TABLE SAMPLE(
                        ID INTEGER PRIMARY KEY AUTOINCREMENT,
                        AGE INT,
                        WORKCLASS TEXT,
                        FNLWGT TEXT,
                        EDUCATION TEXT,
                        EDUCATION_NUM TEXT,
                        MARITAL_STATUS TEXT,
                        OCCUPATION TEXT,
                        RELATIONSHIP TEXT,
                        RACE TEXT,
                        SEX TEXT,
                        CAPITAL_GAIN int,
                        CAPITAL_LOST int,
                        HOURS_PER_WEEK TEXT,
                        NATIVE_COUNTRY TEXT,
                        NET_CAPITAL_GAIN INT); '''

connection.execute(query_create_table)

insert_query = "INSERT INTO SAMPLE( AGE, WORKCLASS, FNLWGT,EDUCATION, EDUCATION_NUM, MARITAL_STATUS, OCCUPATION, RELATIONSHIP, RACE, SEX, CAPITAL_GAIN, CAPITAL_LOST, HOURS_PER_WEEK, NATIVE_COUNTRY) VALUES (%d, '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s')"

for index, row in sqladb.iterrows():
    connection.execute(insert_query % (row['age'], row['workclass'], row['fnlwgt'], row['education'], row['education_num'], row['marital_status'], row['occupation'], row['relationship'], row['race'], row['sex'], row['capital_gain'], row['capital_loss'], row['hours_per_week'], row['native_country']))

connection.commit()

In [99]:
top_10_rows = pd.read_sql_query("select * from SAMPLE limit 10", connection)
print(top_10_rows)

   ID  AGE          WORKCLASS  FNLWGT   EDUCATION EDUCATION_NUM  \
0   1   39          State-gov   77516   Bachelors            13   
1   2   50   Self-emp-not-inc   83311   Bachelors            13   
2   3   38            Private  215646     HS-grad             9   
3   4   53            Private  234721        11th             7   
4   5   28            Private  338409   Bachelors            13   
5   6   37            Private  284582     Masters            14   
6   7   49            Private  160187         9th             5   
7   8   52   Self-emp-not-inc  209642     HS-grad             9   
8   9   31            Private   45781     Masters            14   
9  10   42            Private  159449   Bachelors            13   

           MARITAL_STATUS          OCCUPATION    RELATIONSHIP    RACE  \
0           Never-married        Adm-clerical   Not-in-family   White   
1      Married-civ-spouse     Exec-managerial         Husband   White   
2                Divorced   Handlers-cleane

In [100]:
male_avg_working=pd.read_sql_query("select sex, AVG(hours_per_week) from SAMPLE where sex = ' Male' group by sex", connection)
print(male_avg_working)

     SEX  AVG(hours_per_week)
0   Male            42.428086


In [101]:
education_count=pd.read_sql_query("select education, COUNT(*) from SAMPLE group by education", connection)
print(education_count)

        EDUCATION  COUNT(*)
0            10th       933
1            11th      1175
2            12th       433
3         1st-4th       168
4         5th-6th       333
5         7th-8th       646
6             9th       514
7      Assoc-acdm      1067
8       Assoc-voc      1382
9       Bachelors      5355
10      Doctorate       413
11        HS-grad     10501
12        Masters      1723
13      Preschool        51
14    Prof-school       576
15   Some-college      7291


In [102]:
occupation_count=pd.read_sql_query("select occupation, COUNT(*) from SAMPLE group by occupation", connection)
print(occupation_count)

            OCCUPATION  COUNT(*)
0                    ?      1843
1         Adm-clerical      3770
2         Armed-Forces         9
3         Craft-repair      4099
4      Exec-managerial      4066
5      Farming-fishing       994
6    Handlers-cleaners      1370
7    Machine-op-inspct      2002
8        Other-service      3295
9      Priv-house-serv       149
10      Prof-specialty      4140
11     Protective-serv       649
12               Sales      3650
13        Tech-support       928
14    Transport-moving      1597


In [103]:
relationship_count=pd.read_sql_query("select relationship, COUNT(*) from SAMPLE group by relationship", connection)
print(relationship_count)

      RELATIONSHIP  COUNT(*)
0          Husband     13193
1    Not-in-family      8305
2   Other-relative       981
3        Own-child      5068
4        Unmarried      3446
5             Wife      1568


In [104]:
private_bachelor_masters=pd.read_sql_query("select count(*) from SAMPLE where education=' Masters' and workclass = ' Private' and marital_status != ' Never-married'", connection)
print(private_bachelor_masters)

   count(*)
0       660


In [105]:
age_min_max=pd.read_sql_query("select workclass, MAX(age), MIN(age), AVG(age) from sample group by workclass", connection)
print(age_min_max)

           WORKCLASS  MAX(age)  MIN(age)   AVG(age)
0                  ?        90        17  40.960240
1        Federal-gov        90        17  42.590625
2          Local-gov        90        17  41.751075
3       Never-worked        30        17  20.571429
4            Private        90        17  36.797585
5       Self-emp-inc        84        17  46.017025
6   Self-emp-not-inc        90        17  44.969697
7          State-gov        81        17  39.436055
8        Without-pay        72        19  47.785714


In [106]:
countrywise_min_max=pd.read_sql_query("select native_country, MAX(age), MIN(age) from sample group by native_country", connection)
print(countrywise_min_max)

                 NATIVE_COUNTRY  MAX(age)  MIN(age)
0                             ?        90        17
1                      Cambodia        65        18
2                        Canada        80        17
3                         China        75        22
4                      Columbia        75        18
5                          Cuba        82        21
6            Dominican-Republic        78        18
7                       Ecuador        90        21
8                   El-Salvador        79        17
9                       England        90        17
10                       France        64        20
11                      Germany        74        18
12                       Greece        65        22
13                    Guatemala        66        19
14                        Haiti        63        17
15           Holand-Netherlands        32        32
16                     Honduras        58        18
17                         Hong        60        19
18          

In [107]:
age_distribution=pd.read_sql_query("select native_country, SUM(CASE WHEN age<25 THEN 1 ELSE 0 END) as [0-25],"+
                                   " SUM(case when age between 25 and 50 then 1 else 0 end) as [25-50],"+
                                   " SUM(case when age between 50 and 75 then 1 else 0 end) as [50-75],"+
                                   " SUM(case when age between 75 and 100 then 1 else 0 end) as [75-100] from sample group by native_country", connection)
print(age_distribution)

                 NATIVE_COUNTRY  0-25  25-50  50-75  75-100
0                             ?    68    416    108       4
1                      Cambodia     1     16      3       0
2                        Canada    13     70     39       2
3                         China     5     50     21       1
4                      Columbia     6     40     14       1
5                          Cuba     5     54     36       2
6            Dominican-Republic    14     43     14       1
7                       Ecuador     4     22      2       1
8                   El-Salvador    28     66     14       1
9                       England    11     58     21       1
10                       France     2     21      6       0
11                      Germany    16     93     30       0
12                       Greece     4     11     14       0
13                    Guatemala    16     42      6       0
14                        Haiti     9     24     11       0
15           Holand-Netherlands     0   

In [ ]:
calc_net_cap=pd.read_sql_query("update sample set Net_Capital_Gain=CAPITAL_GAIN+CAPITAL_LOST ", connection)
print(select_table)